<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/main/unet-experiment-1/notebooks/experiment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net Experiement 1: Exploring variance/ randomness

When running the exact same model with the exact same dataset we get slightly different results each time due to randomness and variability involved in the training process.
the variance refers to 
The goal of this notebook is to evaluate the variability in the results obtained from different training runs on the same dataset We will use the exact same setup and dataset as in the unet notebook and execute 10 training runs.

### 0. Helper Classes

In [ ]:
import numpy as np
from tensorflow import keras
import pickle


class EvaluationMetrics:
    """
        This class calculates and summarizes evaluation metrics based on the predicted and true labels.
    """

    def __init__(self, x_train, x_val, x_test, y_train, y_val, y_test, y_pred, dataset, training_dates, validation_dates, testing_dates, tile_size, step_size,
                 run_count):
       
        self.y_pred = y_pred
        if dataset == 'training':
          self.y_true = y_train
        elif dataset == 'validation':
          self.y_true = y_val
        elif dataset == 'testing':
          self.y_true = y_test
        else: 
          raise Exception(f'Specify the dataset to be used for calculating the metrics. The string has to be either "testing", "training" or "validation", however it was {dataset}.')
        
        self.class_statistics = self.get_statistics(x_train, x_val, x_test, y_train, y_val, y_test)

        self.training_dates = training_dates
        self.validation_dates = validation_dates
        self.testing_dates = testing_dates
        self.tile_size = tile_size
        self.step_size = step_size
        self.run_count = run_count

        self.jacard = self.jacard_coef(self.y_true, y_pred)



        self.conf_matrix_land = self.confusion_matrix(self.y_true, y_pred, 2)
        self.conf_matrix_valid = self.confusion_matrix(self.y_true, y_pred, 1)
        self.conf_matrix_invalid = self.confusion_matrix(self.y_true, y_pred, 0)

        self.precision_land = self.precision(self.conf_matrix_land)
        self.sensitivity_recall_land = self.sensitivity_recall(self.conf_matrix_land)
        self.specificy_land = self.specificy(self.conf_matrix_land)

        self.precision_valid = self.precision(self.conf_matrix_valid)
        self.sensitivity_recall_valid = self.sensitivity_recall(self.conf_matrix_valid)
        self.specificy_valid = self.specificy(self.conf_matrix_valid)

        self.precision_invalid = self.precision(self.conf_matrix_invalid)
        self.sensitivity_recall_invalid = self.sensitivity_recall(self.conf_matrix_invalid)
        self.specificy_invalid = self.specificy(self.conf_matrix_invalid)

        self.f1_land = self.f1_scores(self.conf_matrix_land)
        self.f1_invalid = self.f1_scores(self.conf_matrix_invalid)
        self.f1_valid = self.f1_scores(self.conf_matrix_valid)

    def jacard_coef(self, y_true, y_pred):
        y_true_f = keras.backend.flatten(y_true)
        y_pred_f = keras.backend.flatten(y_pred)

        intersection = keras.backend.sum(y_true_f * y_pred_f)
        return (intersection + 1.0) / (
                keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection + 1.0
        )  #todo reason for +1?

    def jacard_rounding_issue(self, y_true, y_pred):
        # revert one hot encoding => binary tensor [0, 0, 1] back to label [2] (3D array to 2D array)
        label_map_true = np.argmax(y_true, axis=-1)
        label_map_pred = np.argmax(y_pred, axis=-1)
        # convert 2D array into 1D array
        flatten_true = np.reshape(label_map_true, (-1,))
        flatten_pred = np.reshape(label_map_pred, (-1,))
        # one hot encoding
        one_hot_true = np.eye(3)[flatten_true]
        one_hot_pred = np.eye(3)[flatten_pred]
        # calculate intersection (A geschnitten B)
        intersection = np.sum(one_hot_true * one_hot_pred)
        # calculate union (a u B, A vereint B)
        union = len(one_hot_true) + len(one_hot_pred) - intersection
        # return jacard coefficient
        return (intersection + 1) / (union + 1)

    def confusion_matrix(self, y_true, y_pred, label):
        true_positives = 0
        false_positives = 0
        true_negatives = 0
        false_negatives = 0

        # revert one hot encoding => binary tensor [0, 0, 1] back to label [2] (3D array to 2D array)
        label_map_true = np.argmax(y_true, axis=-1)
        label_map_pred = np.argmax(y_pred, axis=-1)
        # convert 2D array into 1D array
        flatten_true = np.reshape(label_map_true, (-1,))
        flatten_pred = np.reshape(label_map_pred, (-1,))

        tp_mask = (flatten_true == flatten_pred) & (flatten_true == label)
        true_positives = np.count_nonzero(tp_mask)

        fn_mask = (flatten_true == label) & (flatten_pred != label)
        false_negatives = np.count_nonzero(fn_mask)

        fp_mask = (flatten_true != label) & (flatten_pred == label)
        false_positives = np.count_nonzero(fp_mask)

        tn_mask = (flatten_true != label) & (flatten_pred != label)
        true_negatives = np.count_nonzero(tn_mask)

        return {
            'true_positives': true_positives,
            'false_positives': false_positives,
            'true_negatives': true_negatives,
            'false_negatives': false_negatives
        }

    def precision(self, conf_matrix):
        return conf_matrix['true_positives'] / (conf_matrix['true_positives'] + conf_matrix['false_positives'])

    def sensitivity_recall(self, conf_matrix):
        return conf_matrix['true_positives'] / (conf_matrix['true_positives'] + conf_matrix['false_negatives'])

    def negative_predictive(self, conf_matrix):
        return conf_matrix['true_negatives'] / (conf_matrix['true_negatives'] + conf_matrix['false_negatives'])

    def specificy(self, conf_matrix):
        return conf_matrix['true_negatives'] / (conf_matrix['true_negatives'] + conf_matrix['false_positives'])

    def f1_scores(self, conf_matrix):
        prec = self.precision(conf_matrix)
        recall = self.sensitivity_recall(conf_matrix)
        return 2 * prec * recall / (prec + recall)

    def print_metrics(self):
        print(f'jacard index: {self.jacard} \n')

        print(f'precision_land: {self.precision_land}')
        print(f'precision_valid: {self.precision_valid}')
        print(f'precision_invalid: {self.precision_invalid} \n')

        print(f'recall_invalid_land: {self.sensitivity_recall_land}')
        print(f'recall_invalid_land: {self.sensitivity_recall_valid}')
        print(f'recall_invalid_land: {self.sensitivity_recall_invalid} \n')

        print(f'specificy_invalid_land: {self.specificy_land}')
        print(f'specificy_invalid_valid: {self.specificy_valid}')
        print(f'specificy_invalid_invalid: {self.specificy_invalid} \n')

        print(f'f1_land: {self.f1_land}')
        print(f'f1_invalid: {self.f1_invalid}')
        print(f'f1_valid: {self.f1_valid}')

        print(f'Training dates: {self.training_dates}, validation dates: {self.validation_dates}, testing dates: {self.testing_dates}')
        print(f'Number of run: {self.run_count}, tile_size: {self.tile_size}, step_size: {self.step_size}')

    def save_to_file(self):
        file_name = f'../metrics/{self.tile_size}_{self.step_size}_{self.run_count}.pkl'
        with open(file_name, 'wb') as file:
            pickle.dump(self, file)

    def get_label_count(self, array):
        revert_one_hot = np.argmax(array, (-1))
        flatten = np.reshape(revert_one_hot, (-1))
        unique_vals, counts = np.unique(flatten, return_counts=True)
        label_count = {}
        for val, count in zip(unique_vals, counts):
            label_count[f'{val}'] = count
        return label_count

    def get_statistics(self, x_train, x_val, x_test, y_train, y_val, y_test):
       return {'y_train': self.get_label_count(y_train),
                 'y_val': self.get_label_count(y_val), 'y_test': self.get_label_count(y_test)}
    # todo add pixel accuracy



### 1. Loading + Preparing Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML
! ls

architecture.drawio  evaluation  notebooks     README.md
Copy_of_unet.ipynb   models	 prepare_data  requirements.txt


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    Dropout,
    UpSampling2D
)
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [ ]:
data_directory = "../data_colab/256_200"

y_train  = np.load(os.path.join(data_directory, '2022_06_20.npz'))['y_mask']
x_train  = np.load(os.path.join(data_directory, '2022_06_20.npz'))['x_input']

y_val = np.load(os.path.join(data_directory, '2022_07_10.npz'))['y_mask']
x_val = np.load(os.path.join(data_directory, '2022_07_10.npz'))['x_input']

y_test = np.load(os.path.join(data_directory, '2022_07_25.npz'))['y_mask']
x_test = np.load(os.path.join(data_directory, '2022_07_25.npz'))['x_input']

print(y_train.shape)
print(x_train.shape)

print(y_val.shape)
print(x_val.shape)

print(y_test.shape)
print(x_test.shape)

(1251, 256, 256)
(1251, 256, 256, 5)
(1323, 256, 256)
(1323, 256, 256, 5)
(1258, 256, 256)
(1258, 256, 256, 5)


In [ ]:
def normalizing(X, y):

  print(y.shape)
  y_one_hot =  np.array([tf.one_hot(item, depth=3).numpy() for item in y])
  print(y_one_hot.shape)
  X_normal = X/255
  return X_normal, y_one_hot

In [ ]:
x_train, y_train = normalizing(x_train, y_train)

X_val, y_val = normalizing(x_val, y_val)

x_test, y_test = normalizing(x_test, y_test)


(1251, 256, 256)
(1251, 256, 256, 3)
(1323, 256, 256)
(1323, 256, 256, 3)
(1258, 256, 256)
(1258, 256, 256, 3)


### 2. Compiling the model

In [ ]:
def unet_2d(input_shape, num_classes):

    # Define the input layer
    inputs = Input(input_shape)

    # Downsample layers
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Upsample layers
    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv7)

    # Define the model
    model = Model(inputs=[inputs], outputs=[output])

    return model

In [ ]:
model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
model.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_300 (Conv2D)            (None, 256, 256, 64  2944        ['input_21[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_301 (Conv2D)            (None, 256, 256, 64  36928       ['conv2d_300[0][0]']             
                                )                                                          

### 3. Execute trainigs + saving results

In [ ]:
tile_size = 256
step_size = 200
saving_path = 'experiment_1'
training_dates = '2022_06_20'
validation_dates = '2022_07_10'
testing_dates = '2022_07_25'

In [ ]:
def execute_training(count):
  print(f'Start training number {count}')
  model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
  model.compile(optimizer='adam',
                loss=categorical_crossentropy,
                metrics=['accuracy']) # are weights resetted?

  early_stop = EarlyStopping(monitor='accuracy', patience=5) 

  model_history = model.fit(x=x_train, y=y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stop]) 
  print('training completed')
  
  # saving model
  model_name = f'{tile_size}_{step_size}_run_{count}'
  model.save(f'../models/{saving_path}/model_{model_name}.h5')
  print('saving model completed')

  # saving model history
  with open(f'../models/{saving_path}/history_{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(model_history.history, file_pi)
  print('saving history completed')

  # making predictions
  pred_test = model.predict(x_test)
  pred_val = model.predict(x_val)
  pred_train = model.predict(x_train)
  print('making predictions completed')
  print(pred_test.shape)
  print(pred_val.shape)
  print(pred_train.shape)

  # calculating metrics
  metrics_test = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_test, pred_test, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
  metrics_val = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_val, pred_val, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
  metrics_train = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_train, pred_train, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
  print('calculating metrics completed')

  # saving metrics
  with open(f'../metrics/{saving_path}/metrics_test{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics_test, file_pi)
  with open(f'../metrics/{saving_path}/metrics_val{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics_val, file_pi)
  with open(f'../metrics/{saving_path}/metrics_train{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics_train, file_pi)
  print('saving metrics completed')

  return  {'metrics_test': metrics_test, 'metrics_val': metrics_val, 'metrics_train': metrics_train}

In [ ]:
all_metrics_test = []
all_metrics_val = []
all_metrics_train = []


for i in range(0,10):
  metrics_dict = execute_training(i)
  all_metrics_test.append(metrics_dict['metrics_test'])
  all_metrics_val.append(metrics_dict['metrics_val'])
  all_metrics_train.append(metrics_dict['metrics_train'])

Start training number 0
Epoch 1/100
40/40 [==============================] - 19s 326ms/step - loss: 0.8118 - accuracy: 0.8085 - val_loss: 7.7246 - val_accuracy: 0.9497
Epoch 2/100
40/40 [==============================] - 10s 241ms/step - loss: 0.1311 - accuracy: 0.9656 - val_loss: 7.4357 - val_accuracy: 0.9747
Epoch 3/100
40/40 [==============================] - 10s 241ms/step - loss: 0.0755 - accuracy: 0.9722 - val_loss: 9.0582 - val_accuracy: 0.9754
Epoch 4/100
40/40 [==============================] - 10s 243ms/step - loss: 0.0677 - accuracy: 0.9743 - val_loss: 15.0085 - val_accuracy: 0.9705
Epoch 5/100
40/40 [==============================] - 10s 243ms/step - loss: 0.0693 - accuracy: 0.9743 - val_loss: 7.2937 - val_accuracy: 0.9773
Epoch 6/100
40/40 [==============================] - 10s 242ms/step - loss: 0.0611 - accuracy: 0.9765 - val_loss: 8.5024 - val_accuracy: 0.9768
Epoch 7/100
40/40 [==============================] - 10s 242ms/step - loss: 0.0576 - accuracy: 0.9785 - val_los

KeyboardInterrupt: ignored

### 4. Results

In [ ]:
for idx, metric in enumerate(all_metrics_test):
  print(f'========= RUN {idx + 1} ============')
  print(f'TEST DATA')
  metric.print_metrics()
  print()
  print(f'VALIDATION DATA')
  all_metrics_val[idx].print_metrics()
  print(f'TRAINING DATA')
  all_metrics_train[idx].print_metrics()
  print()
  print()



In [ ]:
def get_mean_jacard(all_metrics):
  jacard_array = []
  for idx, metric in enumerate(all_metrics):
    print(metric.jacard)
    jacard_array.append(metric.jacard)

  print()
  print(f'Mean jacard index: {sum(jacard_array)/10}')
  print()
  print(f'Worst index: {min(jacard_array)}')
  print(f'Best index: {max(jacard_array)}')
  print(f'Variance: {max(jacard_array)-min(jacard_array)}')

print('============ TEST DATA ===================')
get_mean_jacard(all_metrics_test)
print()

print('============ VALIDATION DATA ===================')
get_mean_jacard(all_metrics_val)
print()

print('============ TRAINING DATA ===================')
get_mean_jacard(all_metrics_train)
print()

execution time: ~ 70 min

tf.Tensor(0.9303678, shape=(), dtype=float32)

tf.Tensor(0.9597043, shape=(), dtype=float32)

tf.Tensor(0.9749496, shape=(), dtype=float32)

tf.Tensor(0.9310194, shape=(), dtype=float32)

tf.Tensor(0.9781205, shape=(), dtype=float32)

tf.Tensor(0.9540997, shape=(), dtype=float32)

tf.Tensor(0.9542077, shape=(), dtype=float32)

tf.Tensor(0.95823056, shape=(), dtype=float32)

tf.Tensor(0.9830309, shape=(), dtype=float32)

tf.Tensor(0.9656351, shape=(), dtype=float32)


Mean jacard index: 0.9589365720748901


Worst index: 0.9303678274154663

Best index: 0.9830309152603149

Variance: 0.05266308784484863


start: 17:14
